# Usage on Windows
## Prerequisites
- Install Anaconda (in Software Center on UH computers).
- Download this notebook and the invoice from Open IRIS in a folder.
- Start Menu -> Anaconda -> Anaconda shell.
- In the shell: "cd" to the folder with the notebook, e.g. "cd Documents\billing_check".
- In the shell: "jupyter notebook". This chould open the notebook in a browser.

## Run the notebook
- Change the invoice file name in the first notebook cell.
- Cell -> Run All.
- Wait for .xslx files to appear in the folder.
- 1st run creates InvoiceNN__with_owners.xlsx file.

## Check cell output
The following cells run test that produce results in .xlsx files:
- Manual interventions
- Check totals before any fixes
- Save test files after fixes

The cell output shows the (rows,cols) count of the test result. If there are more than 0 rows, you can check the .xlsx.

## Manual corrections
You can use the test .xlsx files as starting point for corrections. This is more convenient than editing the complete invoice from IRIS. The changes can be merged to the complete invoice with the notebook merge_xlsx.ipynb.

As an example, let's say you want to add missing price types:
- Open test_InvoiceNN__price_type_missing.xlsx.
- Make corrections.
- Save as fixed_InvoiceNN__price_type_missing.xlsx (to prevent corrections being over-written).
- Run merge_xlsx.ipynb with FIXES_FILE = 'fixed_InvoiceNN__price_type_missing.xlsx' and INVOICE_FILE = 'InvoiceNN__with_owners.xlsx'.
- Run this notebook from the beginning. Do not change invoice from the first run.

merge_xlsx.ipynb creates timestamped versions of the invoice. To undo manual corrections, delete the corresponding versions from the invoice folder.

## Add product purchases
As the final step, you can append product purchases to the invoice. This can be done with merge_xlsx notebook. Please note that as the product rows do not contain most columns in the invoice, the resulting .xlsx will not work in this notebook. 


# Connect to Google Drive from Colaboratory

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

# Price list and invoice
- set invoice file
- set holidays

In [ ]:
# '.' works on local computer when the price list is in the notebook folder
PRICE_LIST_DIR = '.' 

# comment out price list you don't want to use
#PRICE_LIST_FILE = 'price_list_biu.csv' 
PRICE_LIST_FILE = 'price_list_lmu.csv' 

import pandas as pd
from pathlib import Path
INVOICE_DIR = './data/LMU/39' 
#INVOICE_DIR = 'C:\\Users\\hajaalin\\Downloads\\LMU39'
INVOICE_NAME = 'Invoice39.xlsx'
REQUESTS_NAME = 'Light Microscopy Unit-provider-requests.xlsx'

# If you want bookings by BIU and/or LMU to appear in the fixed invoice, 
# set the flags below to True. Otherwise they will be filtered out and will
# appear only in 'test_InvoiceNN__[BIU,LMU]_bookings.xlsx'
INVOICE_BIU_GROUPS = True
INVOICE_LMU_GROUPS = False


from utils import find_latest_invoice_version

PRICE_LIST_DIR = Path(PRICE_LIST_DIR)
if not PRICE_LIST_DIR.is_dir():
    raise ValueError('Please check PRICE_LIST_DIR.')
PRICE_LIST_FILE = PRICE_LIST_DIR / PRICE_LIST_FILE
if not PRICE_LIST_FILE.exists():
    raise ValueError('Please check PRICE_LIST_FILE.')

INVOICE_DIR = Path(INVOICE_DIR)
if not INVOICE_DIR.is_dir():
    raise ValueError('Please check INVOICE_DIR.')
INVOICE_FILE = INVOICE_DIR / INVOICE_NAME
if not INVOICE_FILE.exists():
    raise ValueError('Please check INVOICE_NAME.')
REQUESTS_FILE = INVOICE_DIR / REQUESTS_NAME
if not REQUESTS_FILE.exists():
    raise ValueError('Please check REQUESTS_NAME.')

DEBUG_DIR = INVOICE_DIR / 'debug'
DEBUG_DIR.mkdir(exist_ok=True)

basename = Path(INVOICE_NAME).stem
ext = Path(INVOICE_NAME).suffix

# read first two rows of input
header = pd.read_excel(INVOICE_FILE, nrows=1)

# check if the first two rows are the invoice summary from IRIS
if 'Created by' in header.columns:
    header = header[header.columns.drop(list(header.filter(regex='Unnamed')))]
    header.to_excel(INVOICE_DIR / (basename + "__header.xlsx"), index=False)
    df = pd.read_excel(INVOICE_FILE, skiprows=[0,1])
else:
    header = pd.DataFrame() # use empty dataframe as marker
    df = pd.read_excel(INVOICE_FILE)
    
# add project owner columns from project list
# this should be done only once, so if the file exists, don't redo
INVOICE_WITH_OWNERS = INVOICE_DIR / (basename + "__with_owners.xlsx")
COL_REQUEST_ID = 'Request ID'
COL_REQUESTER_EMAIL = 'Requester email'
COL_REQUESTER_NAME = 'Requester name'
if INVOICE_WITH_OWNERS.exists():
    INVOICE_WITH_OWNERS = find_latest_invoice_version(INVOICE_WITH_OWNERS)
    df = pd.read_excel(INVOICE_WITH_OWNERS)
else:
    req = pd.read_excel(REQUESTS_FILE)
    req = req[[COL_REQUEST_ID, COL_REQUESTER_EMAIL, COL_REQUESTER_NAME]]
    df = pd.merge(df, req, on=COL_REQUEST_ID)
    df.to_excel(INVOICE_WITH_OWNERS, index=False)

print(INVOICE_WITH_OWNERS)


In [ ]:
from datetime import date,datetime,timedelta
import numpy as np

# holidays during billing period (have to list manually)
holidays = [\
    date(2019,12,6), \
    date(2019,12,24), \
    date(2019,12,25), \
    date(2019,12,26), \
    date(2020,1,1), \
    date(2020,1,6), \
    date(2020,4,10), \
    date(2020,4,13), \
    date(2020,5,1), \
    date(2020,5,21), \
    date(2020,6,19), \
    date(2020,12,6), \
    date(2020,12,24), \
    date(2020,12,25), \
    date(2020,12,26), \
    date(2021,1,1), \
    date(2021,1,6), \
    date(2021,4,2), \
    date(2021,4,5), \
    date(2021,5,1), \
    date(2021,5,13), \
    date(2021,6,25), \
    date(2021,12,6), \
    date(2021,12,24), \
    date(2021,12,25), \
    date(2021,12,26), \
    date(2022,1,1), \
    date(2022,1,6), \
    date(2022,4,15), \
    date(2022,4,18), \
    date(2022,5,1), \
    date(2022,5,26), \
    date(2022,6,24), \
    date(2022,12,6), \
    date(2022,12,24), \
    date(2022,12,25), \
    date(2022,12,26), \
    date(2023,1,1), \
    date(2023,1,6), \
    date(2023,4,6), \
    date(2023,4,10), \
    date(2023,5,1), \
    date(2023,5,18), \
    date(2023,6,23), \
           ]


RESOURCE = 'Resource/Product'
# essential columns shown in testing
summary = ['ID','User name',RESOURCE,'Booking start','Booking end', 'Quantity','Price type','Charge','Group','Cost center name','Cost center code','Discount', 'Comments (charge)','Price (detailed)']
summary_long = ['ID','User name',RESOURCE,'Booking start','Booking end','Price','Discount', 'Quantity','Price item','Price type','Charge','Group','Cost center name','Cost center code', 'Comments (charge)','Price (detailed)']
summary_short = ['ID','User name',RESOURCE,'Booking start','Booking end','Quantity','Charge','Discount', 'Comments (charge)','Group','Cost center name','Cost center code']


# exclude IRIS test instruments
test_instruments = ['Sheep (TESTING AND DEVELOPMENT PURPOSE', 'Super testers practice instrument']
df = df[~df[RESOURCE].isin(test_instruments)]

# exclude (first save) staff groups
biu_groups = ['BIU staff', 'BIU Task test']
lmu_groups = ['Tanhuanpää Kimmo', 'LMU-staff']
test_groups = ['Group Raimi research inc']

cols = summary.copy()
cols.append('Booking title')
cols.append('Booking comments')
df[df['Group'].isin(biu_groups)][cols].to_excel(INVOICE_DIR / ("tmp_" + basename + "__BIU_bookings.xlsx"), index=True)
df[df['Group'].isin(lmu_groups)][cols].to_excel(INVOICE_DIR / ("tmp_" + basename + "__LMU_bookings.xlsx"), index=True)

exclude_groups = []
if not INVOICE_BIU_GROUPS:
    exclude_groups.extend(biu_groups)
if not INVOICE_LMU_GROUPS:
    exclude_groups.extend(lmu_groups)
exclude_groups.extend(test_groups)
df = df[~df['Group'].isin(exclude_groups)]

# store original report with essential columns
df[summary].to_excel(DEBUG_DIR / ("tmp_" + basename + "__summary" + ext), index=True) 


In [ ]:
header

In [ ]:
df[df['Waived']==True][summary]

# Manual interventions
This cell saves rows that might need manual editing and test cases that are not expected
to be changed by the automated fixes below.

You can make the manual edits in the shorter .xlsx files produced here, and then merge the changes to the original IRIS report using the notebook "merge_reports.ipynb".

In [ ]:
PRICE_TYPE = 'Price type'
PRIME_TIME = 'Prime-time'
OFF_HOURS = 'Off-hours'
NIGHT_TIME = 'Night time'
CANCELLATION_FEE = 'Cancellation fee'
TRAINING_FEE = "Training fee"
REBOOKED = 'Rebooked'

def save_test_result(filename,dataframe,where=INVOICE_DIR):
    print(filename + " (rows, cols) " + str(dataframe.shape))
    dataframe.to_excel(where / filename, index=True)

# save rows with price type missing
test = df[df[PRICE_TYPE].isnull()]
save_test_result("test_" + basename + "__price_type_missing.xlsx", test[summary])

# save rows with request ID missing
test = df[df['Request ID'].isnull()]
save_test_result("test_" + basename + "__request_id_missing.xlsx", test[summary])

# save rows with request ID missing
test = df[df[COL_REQUESTER_NAME].isnull()]
save_test_result("test_" + basename + "__requester_name_missing.xlsx", test[summary], where=DEBUG_DIR)

# group or WBS missing
test = df[(df['Group'].isnull()) | (df['Cost center code'].isnull())][summary_short]
save_test_result("test_" + basename + '__group_or_wbs_missing.xlsx', test)

# remit code missing
test = df[df['Remit code'].isnull()][summary_short]
save_test_result("test_" + basename + '__remit_code_missing.xlsx', test)

# save rows with cancellations
test = df[df['Price item'].str.contains("Cancellation")]
save_test_result("test_" + basename + "__cancellations.xlsx", test[summary_short], where=DEBUG_DIR)

# save rows with commented cancellations
test = df[(df['Price item'].str.contains("Cancellation")) & (~df['Booking comments'].isnull())]
cols = summary_short.copy()
cols.append('Booking comments')
save_test_result("test_" + basename + "__cancellation_reasons.xlsx", test[cols])

# save rows with discounts
test = df[~df['Discount'].isnull()]
save_test_result("test_" + basename + "__discount.xlsx", test[summary_short], where=DEBUG_DIR)

# save rows with waived charges
test = df[df['Waived']==True]
save_test_result("test_" + basename + "__waived.xlsx", test[summary], where=DEBUG_DIR)

# save rows with no PI email
test = df[df['Group head(s) text'].isnull()]
cols = summary_short.copy()
cols.append('Group head(s) text')
save_test_result("test_" + basename + "__pi_email_missing.xlsx", test[cols])


# find product rows
if not 'tmp_is_product' in df.columns:
    df['tmp_is_product'] = df['Charge type']=='Product (request)'
products = df[df['tmp_is_product']].copy()

# fix product rows in main invoice (to avoid missing price)
for i in products.index:
    df.at[i,RESOURCE] = df.loc[i]['Price list/Product']

# remove whitespace
df[RESOURCE] = df[RESOURCE].astype(str)
df[RESOURCE] = df[RESOURCE].str.strip()

# find proper values for product name, user who bought the product, purchase date
products[RESOURCE] = products['Price list/Product']
products['User name'] = products['Product comments']
products['Booking start'] = products['Product purchase date'] ## TODO: 'Charge date'?
products['Booking end'] = products['Product purchase date']

# save product rows
cols = ['ID','Product comments',RESOURCE,'Booking start','Booking end','Quantity','Charge','Discount', 'Comments (charge)','Group','Cost center name','Cost center code']
save_test_result("test_" + basename + "__products.xlsx", products[cols], where=INVOICE_DIR)





# Read price list

In [ ]:
df[RESOURCE].unique()

In [ ]:
prices = pd.read_csv(PRICE_LIST_FILE, quotechar="'",)

print('Price types in use:')
print(df[PRICE_TYPE].unique())

# Add price item for training, with same price as PRIME_TIME
prices[TRAINING_FEE] = prices[PRIME_TIME]

# Check that prices exist for all instruments
for r in df[RESOURCE].unique():
    for pt in df[PRICE_TYPE].unique():
        for p in [PRIME_TIME,OFF_HOURS,NIGHT_TIME]:
            try:
                mask = (prices.Instrument == r) & (prices[PRICE_TYPE] == pt)
                #print("'%s'" % r)
                price = prices[mask].values[0]
            except:
                raise ValueError("Price missing: %s / %s / %s" % (r,pt,p))


# Functions and constants

In [ ]:
# read these columns as datetime
df['Booking start'] =  pd.to_datetime(df['Booking start'], format='%Y-%m-%d %H:%M')
df['Booking end'] =  pd.to_datetime(df['Booking end'], format='%Y-%m-%d %H:%M')

# force wbs codes to be string
df['Cost center code'] =  df['Cost center code'].astype(str)
products['Cost center code'] =  products['Cost center code'].astype(str)

# billing period
period_start = df['Booking start'].min().date()
period_end = df['Booking end'].max().date()

# weekends during billing period
weekends = []
test = period_start
while test < period_end:
    if test.weekday() >= 5:
        weekends.append(test)
    test = test + timedelta(days=1)
weekends

weekends_and_holidays = sorted(weekends + holidays)

instruments_with_night_discounts = prices[~prices[NIGHT_TIME].isnull()]['Instrument'].unique()
instruments_with_no_discounts = prices[(prices[PRIME_TIME] == prices[OFF_HOURS]) & \
                                             (prices[NIGHT_TIME].isnull())]['Instrument'].unique()
instruments_with_cancellation_fee = prices[~prices[CANCELLATION_FEE].isnull()]['Instrument'].unique()
print('instruments_with_night_discounts')
print(instruments_with_night_discounts)
print('instruments_with_no_discounts')
print(instruments_with_no_discounts)
print('instruments_with_cancellation_fee')
print(instruments_with_cancellation_fee)


def next_workday(dt):
    #print(weekends_and_holidays)
    test = dt
    while True:
        test = test + timedelta(days=1)
        #print(test.date())
        if test.date() not in weekends_and_holidays:
            return test

# get price type for a booking that has already been split
def get_price_item(row):
    if (row['tmp_cancellation'] == True):
        if (row['tmp_rebooked'] == True):
            return REBOOKED
        else:
            return CANCELLATION_FEE
        
    if "Training" in row['tmp_price_item_iris']:
        return TRAINING_FEE

    if row[RESOURCE] in instruments_with_night_discounts:
        if row['Booking start'].hour < 8 or row['Booking start'].hour >= 22:
            return NIGHT_TIME
        if includes_holiday_or_weekend(row) or row['Booking start'].hour == 8 or row['Booking start'].hour >= 17:
            return OFF_HOURS
        else:
            return PRIME_TIME
    else:
        if includes_holiday_or_weekend(row) or row['Booking start'].hour < 9 or row['Booking start'].hour >= 17:
            return OFF_HOURS
        else:
            return PRIME_TIME
        
def get_price(row):
    #print(row[[RESOURCE,PRICE_TYPE]])
    #print(get_price_item(row))
    if get_price_item(row) == REBOOKED:
        return 0

    return prices[(prices['Instrument'] == row[RESOURCE]) & \
                  (prices[PRICE_TYPE] == row[PRICE_TYPE])][get_price_item(row)].values[0]

def get_discount_factor(row):
    discount = row['Discount']
    discount = discount.replace('%','')
    if(discount == 'nan'):
        return 1
    else:
        return 1 - float(discount)/100

df['Discount'] = df['Discount'].astype(str)
df['tmp_discount_factor'] = 1
df['tmp_discount_factor'] = df.apply(get_discount_factor, axis=1)
test = df[df['tmp_discount_factor'] != 1]
cols = summary_short.copy()
cols.append('tmp_discount_factor')
save_test_result("test_" + basename + "__discount_factor.xlsx", test[cols], where=DEBUG_DIR)

# splits between prime and off hours (no night discount)
def next_split_2(dt):
    # other to prime time
    if dt.hour >= 17:
        nwd = next_workday(dt)
        return datetime(nwd.year,nwd.month,nwd.day,9,0,0)
    if dt.hour < 9:
        if dt.date() not in weekends_and_holidays:
            return datetime(dt.year,dt.month,dt.day,9,0,0)
        else:
            nwd = next_workday(dt)
            return datetime(nwd.year,nwd.month,nwd.day,9,0,0) 
    # prime to other time
    if dt.hour < 17:
        if dt.date() not in weekends_and_holidays:
            return datetime(dt.year,dt.month,dt.day,17,0,0)
        else:
            nwd = next_workday(dt)
            return datetime(nwd.year,nwd.month,nwd.day,9,0,0) 
    
# splits between prime, off and night hours 
def next_split_3(dt):
    # night to other time
    if dt.hour >= 22:
        return datetime(dt.year,dt.month,dt.day,8,0,0) + timedelta(days=1)
    if dt.hour < 8:
        return datetime(dt.year,dt.month,dt.day,8,0,0)
    # other to prime time (or night time during holidays)
    if dt.hour < 9:
        if (dt.date() in weekends_and_holidays):
            return datetime(dt.year,dt.month,dt.day,22,0,0)
        else:
            return datetime(dt.year,dt.month,dt.day,9,0,0)
    # prime to other time (or night time during holidays)
    if dt.hour < 17:
        if (dt.date() in weekends_and_holidays):
            return datetime(dt.year,dt.month,dt.day,22,0,0)
        else:
            return datetime(dt.year,dt.month,dt.day,17,0,0)
    # other to night time
    if dt.hour < 22:
        return datetime(dt.year,dt.month,dt.day,22,0,0)
    
# does a row need prime/off split
def needs_split_2(row):
    if row[RESOURCE] in np.append(instruments_with_night_discounts,instruments_with_no_discounts):
        return False
    if next_split_2(row['Booking start']) < row['Booking end']:
        #print('start ' + str(row['Booking start']))
        #print('next ' + str(next_split_2(row['Booking start'])))
        #print('end ' + str(row['Booking end']))
        #print("yes")
        return True
    else:
        return False

# does a row need prime/off/night split
def needs_split_3(row):
    if not row[RESOURCE] in instruments_with_night_discounts:
        return False
    if next_split_3(row['Booking start']) < row['Booking end']:
        #print('start ' + str(row['Booking start']))
        #print('next ' + str(next_split_3(row['Booking start'])))
        #print('end ' + str(row['Booking end']))
        #print("yes")
        return True
    else:
        return False

def night_discount_applies(row):
    if row[RESOURCE] in instruments_with_night_discounts:
        return True
    else:
        return False

def off_hour_discount_applies(row):
    if row[RESOURCE] in instruments_with_no_discounts:
        return False
    else:
        return True
    
def cancellation_fee_applies(row):
    if row[RESOURCE] in instruments_with_cancellation_fee:
        return True
    else:
        return False

def is_night_1(row):
    if not night_discount_applies(row):
        return False
    #print(row.head())
    dt1 = row['Booking start']
    dt2 = row['Booking end']
    if dt2 > datetime(dt1.year, dt1.month, dt1.day, 22, 0, 0):
        return True
    else:
        return False

def is_night_2(row):
    if not night_discount_applies(row):
        return False
    #print(row.head())
    dt1 = row['Booking start']
    dt2 = row['Booking end']
    if (dt1.day < dt2.day) or (dt1.hour < 8):
        return True
    else:
        return False

def is_cancellation(row):
    if "Cancellation" in row['Price item']:
        return True
    else:
        return False

def includes_holiday_or_weekend(row):
    d1 = row['Booking start'].date()
    d2 = row['Booking end'].date()

    for dt in weekends_and_holidays:
        if (d1 <= dt) and (d2 >= dt):
            return True
        
    return False

def calculate_duration(row):
    delta = row['Booking end'] - row['Booking start']
    return round(delta.total_seconds() / 3600, 4)


def calculate_charge(row):
    # leave cancellations as they are
    if row['tmp_cancellation_fee_applies'] and row['tmp_cancellation']:
        return row['Charge']
    
    if row['Waived'] == True:
        return 0
    
    return round(row['Quantity'] * row['Price'] * row['tmp_discount_factor'], 2)


def split_3(rows):
    _summary = ['ID','Booking start','Booking end','Price']

    # make a copy, just to be sure
    rows = rows.copy()
    #print("rows:")
    #print(rows[_summary])
    
    rows['Booking start'] =  pd.to_datetime(rows['Booking start'], format='%Y-%m-%d %H:%M')
    rows['Booking end'] =  pd.to_datetime(rows['Booking end'], format='%Y-%m-%d %H:%M')

    tmp = rows.tail(1).copy()
    #print("tmp:")
    #print(tmp[_summary])
    tmp['Booking start'] =  pd.to_datetime(tmp['Booking start'], format='%Y-%m-%d %H:%M')
    tmp['Booking end'] =  pd.to_datetime(tmp['Booking end'], format='%Y-%m-%d %H:%M')
    i = tmp.index.values[0]
    #print(i)
    next_split = next_split_3(tmp.loc[i]['Booking start'])
    
    if tmp.loc[i]['Booking end'] <= next_split:
        #print("return split")
        rows['Price item'] = rows.apply(get_price_item, axis=1)
        rows['Price'] = rows.apply(get_price, axis=1)
        rows['Quantity'] = rows.apply(calculate_duration, axis=1)
        rows['Charge'] = rows.apply(calculate_charge, axis=1)

        return rows
    
    else:
        rows.at[i,'Booking end'] = next_split
        
        tmp.at[i,'Booking start'] = next_split
        rows = rows.append(tmp)
        rows.reset_index(inplace=True, drop=True)

        return split_3(rows)
    
from utils import check_totals


# Check totals before any fixes

In [ ]:
check_totals(df,'before_fixes',DEBUG_DIR,basename)

# Split invoice to bookings and products

In [ ]:
# remove products from main invoice
df = df[~df['tmp_is_product']]


# Add missing discounts


In [ ]:
cols = summary_short.copy()
cols.append('Booking ID')
cols.append('tmp_discount_factor')

# find split lines with discounts
test = df[(df.duplicated(subset=['Booking ID'], keep=False)) & (df['tmp_discount_factor'] != 1)]
save_test_result("test_" + basename + "__discount_with_split.xlsx", test[cols], where=DEBUG_DIR)

for i in test.index:
    bookingID = df.loc[i]['Booking ID']
    discount = df.loc[i]['Discount']
    discount_factor = df.loc[i]['tmp_discount_factor']
    
    # find all lines with this bookingID
    for j in df[df['Booking ID'] == bookingID].index:
        # apply the same discount
        df.at[j,'Discount'] = discount
        df.at[j,'tmp_discount_factor'] = discount_factor

test = df[(df.duplicated(subset=['Booking ID'], keep=False)) & (df['tmp_discount_factor'] != 1)]
save_test_result("test_" + basename + "__discount_with_split_fixed.xlsx", test[cols], where=DEBUG_DIR)


# Apply price list
Overwrite IRIS prices with the .csv price list. Recalculate charges assuming that bookings are split (apply price item based on start time). 

Note: this will not be needed once IRIS has night prices.

In [ ]:
# store original price and charge in a new columns
df['tmp_price_item_iris'] = df['Price item']
df['tmp_price_iris'] = df['Price']
df['tmp_quantity_iris'] = df['Quantity']
df['tmp_charge_iris'] = df['Charge']

# store info about cancellations. 
df['tmp_cancellation_fee_applies'] = df.apply(cancellation_fee_applies, axis=1)
df['tmp_cancellation'] = df.apply(is_cancellation, axis=1)
# this flag can be set manually to indicate (parts of) cancelled bookings that have been rebooked.
if 'tmp_rebooked' not in df.columns:
    print('column tmp_rebooked does not exist, creating...')
    df['tmp_rebooked'] = False

df['Price item'] = df.apply(get_price_item, axis=1)
df['Price'] = df.apply(get_price, axis=1)

## Recalculating durations and charges may be useful if merging data with manually added rows.
df['Quantity'] = df.apply(calculate_duration, axis=1)
df['Charge'] = df.apply(calculate_charge, axis=1)

check_totals(df,'after_recalculate_charges',DEBUG_DIR,basename)

# save training bookings
cols = summary_short.copy()
cols.append('tmp_price_item_iris')
test = df[df['tmp_price_item_iris'].str.contains("Training")]
save_test_result("test_" + basename + "__trainings.xlsx", test[cols], where=DEBUG_DIR)

# save rows where duration changes
cols = summary_short.copy()
cols.append('tmp_quantity_iris')
test = df[df['Quantity'] != df['tmp_quantity_iris']]
save_test_result("test_" + basename + "__duration_changed.xlsx", test[cols], where=DEBUG_DIR)

# save rows where charge changes
df['tmp_charge_diff'] = df['Charge'] - df['tmp_charge_iris']
cols = summary_short.copy()
cols.append('tmp_charge_iris')
cols.append('tmp_charge_diff')
# ignore differences under 1 EUR.
test = df[df['tmp_charge_diff'].abs() > 1]
save_test_result("test_" + basename + "__charge_changed.xlsx", test[cols], where=DEBUG_DIR)


In [ ]:
df.columns

# Test splitting

In [ ]:
SPLIT_TEST_INDEX = -1
#SPLIT_TEST_INDEX = 11849

if SPLIT_TEST_INDEX != -1:
    # test split
    i = df[df['ID'] == 11849].index.values[0]
    #i = df[df['ID'] == 12954].index.values[0]
    dup = pd.DataFrame(df.loc[i]).T
    #dup = pd.DataFrame(df[df['ID'] == 11849]).T
    print(dup[summary])
    print()
    dup.index.values[0]
    split_3(dup)[summary]

# Add flag columns to indicate particular situations

In [ ]:
# add columns that help to find particular situations
df['tmp_needs_split_2'] = df.apply(needs_split_2, axis=1)
df['tmp_needs_split_3'] = df.apply(needs_split_3, axis=1)
df['tmp_is_night_1'] = df.apply(is_night_1, axis=1)
df['tmp_is_night_2'] = df.apply(is_night_2, axis=1)
df['tmp_night_discount_applies'] = df.apply(night_discount_applies, axis=1)
df['tmp_off_hour_discount_applies'] = df.apply(off_hour_discount_applies, axis=1)
df['tmp_includes_holiday_or_weekend'] = df.apply(includes_holiday_or_weekend, axis=1)



# Search problematic cases and save results

In [ ]:
def run_tests(tag):
    prefix = "test_" + basename + "_" + tag + "_"
    # missing a prime/off/night split
    test = df[df['tmp_needs_split_3']][summary]
    save_test_result(prefix + '_needs_split_3.xlsx', test, where=DEBUG_DIR)

    # missing a prime/off split
    test = df[df['tmp_needs_split_2']][summary]
    save_test_result(prefix + '_needs_split_2.xlsx', test, where=DEBUG_DIR)

    # booking includes both night discount periods
    test = df[df['tmp_is_night_1'] & df['tmp_is_night_2']][summary]
    save_test_result(prefix + '_night1_and_night2.xlsx', test, where=DEBUG_DIR)

    # both night discounts and a missing split
    test = df[df['tmp_is_night_1'] & df['tmp_is_night_2'] & df['tmp_needs_split_3']][summary]
    save_test_result(prefix + '_night1_and_night2_needs_split3.xlsx', test, where=DEBUG_DIR)

    # find regular price bookings during holidays
    test = df[df['tmp_includes_holiday_or_weekend'] & df['Price item'].str.startswith('Regular usage') & df['tmp_off_hour_discount_applies']][summary]
    save_test_result(prefix + '_regular_price_during_holidays.xlsx', test, where=DEBUG_DIR)


run_tests("_before_fixes")

## Split bookings

In [ ]:
# bookings with missing splits
index = df[df['tmp_needs_split_3']].index
#print(index)

df['tmp_remove_split_line'] = False

# dataframe for split bookings
splits = pd.DataFrame()
for i in index:
    #print(i)

    y1 = df.loc[i]['Booking start'].year
    m1 = df.loc[i]['Booking start'].month
    d1 = df.loc[i]['Booking start'].day

    #print(df.loc[i]['ID']) 
    # create a new DataFrame from the line to be split
    dup = pd.DataFrame(df.loc[i]).T
    
    # split the line
    split = split_3(dup)
    #print(split[summary])
    #print(split.shape)
    
    # mark original line for removal
    df.at[i,'tmp_remove_split_line'] = True
    
    # append the split line to the dataframe with all splits 
    splits = splits.append(split, ignore_index=True)
    
# append all splits to the original dataframe
df = df.append(splits, ignore_index=True)

# save splits including the original line 
df[df['tmp_needs_split_3'] == True] \
    .sort_values(['ID','Booking start','Quantity'], ascending=[True, True, False])[summary] \
    .to_excel(DEBUG_DIR / ("tmp_" + basename + "__fix1_split" + ext), index=True) 

# remove original line and save dataframe
df = df[df['tmp_remove_split_line'] == False]
df[summary].to_excel(DEBUG_DIR / ("tmp_" + basename + "__fix1_split_originals_removed" + ext), index=True) 


## Fix holidays

In [ ]:
# fix holidays
idx = df[df['tmp_includes_holiday_or_weekend'] & ~df['Price item'].str.startswith('Night') & df['tmp_off_hour_discount_applies']].index
for i in idx:
    #print(df.loc[i]['ID']) 
    # by eyeballing the list it seems there is no need for splitting, so it's enough to edit the price
    df.at[i,'Price item'] = OFF_HOURS
    #print(df.loc[i][summary].T)
    #print(prices[prices['Instrument'] == df.loc[i][RESOURCE]])
    #print(prices[prices['Instrument'] == df.loc[i][RESOURCE]]['Off-hours'])
    df.at[i,'Price'] = get_price(df.loc[i]) 
    df.at[i,'Charge'] = calculate_charge(df.loc[i])
    
df.loc[idx][summary].to_excel(DEBUG_DIR / ("tmp_" + basename + "__fix2_holidays" + ext), index=True) 


## Fix night time discounts

In [ ]:
# fix night time reservations that didn't need a split but include both discounts
idx = df[df['tmp_is_night_1'] & df['tmp_is_night_2'] & ~df['tmp_needs_split_3']][summary].index
for i in idx:
    # if booking starts after 22, it's enough to fix price
    if(df.loc[i]['Booking start'].hour >= 22):
        print("fix price and charge for " + str(df.loc[i]['ID']))
        df.at[i,'Price item'] = NIGHT_TIME
        df.at[i,'Price'] = get_price(df.loc[i])
        df.at[i,'Charge'] = calculate_charge(df.loc[i])
    else:
        print("do nothing, this should have been split " + str(df.loc[i]['ID']))

cols = summary_short.copy()
cols.append('tmp_charge_iris')
cols.append('tmp_charge_diff')

df.loc[idx][cols].to_excel(DEBUG_DIR / ("tmp_" + basename + "__fix3_nights_1_and_2" + ext), index=True) 


In [ ]:
df['tmp_num_issue'] = ~df["Price"].apply(np.isreal)
df[df['tmp_num_issue']==True]

# Find overlapping bookings 

In [ ]:
INTERVAL = 'tmp_booking_interval'
OVERLAPS = 'tmp_overlapping_bookings'
HAS_OVERLAPS = 'tmp_has_overlapping_bookings'

df[OVERLAPS] = ''
df[HAS_OVERLAPS] = False

def booking_interval(row):
    return pd.Interval(left=row['Booking start'], right=row['Booking end'])

def overlaps_another_booking(row, testrow):
    overlaps = row[OVERLAPS]
    if (row[RESOURCE] == testrow[RESOURCE]) \
        & (row['ID'] != testrow['ID']) \
        & (row[INTERVAL].overlaps(testrow[INTERVAL])):
        overlaps = overlaps + ',' + str(testrow['ID'])    
    return overlaps

def find_overlaps(ia):
    overlaps = []
    
    tuples = ia.to_tuples()
    #print(tuples)
    for i in range(1,len(tuples)-1):
        if tuples[i][0] < tuples[i-1][1]:
            #print(tuples[i-1])
            #print(tuples[i])
            overlaps.append(pd.Interval(*tuples[i-1]))
            overlaps.append(pd.Interval(*tuples[i]))
    
    return overlaps

df[INTERVAL] = df.apply(booking_interval, axis=1)

# Sort bookings by resource and booking period. This allows to use IntervalArray.is_non_overlapping_monotonic below.
df = df.sort_values([RESOURCE,'Booking start','Booking end'], ascending=[True, True, True])

intervals = {}
for r in df[RESOURCE].unique():
    dfr = df[df[RESOURCE] == r]
    ia = pd.arrays.IntervalArray(dfr[INTERVAL].values)
    intervals[r] = ia
    
    if not ia.is_non_overlapping_monotonic:
        print(r + " has overlapping bookings")
        
        overlaps = find_overlaps(ia)
        #print(overlaps)
        
        idx = dfr[dfr[INTERVAL].isin(overlaps)].index
        for i in idx:
            df.at[i, HAS_OVERLAPS] = True
            #print(i)

# save bookings with overlaps
cols = ['ID','User name',RESOURCE,'Booking start','Booking end','Quantity','tmp_cancellation','tmp_cancellation_fee_applies','tmp_rebooked',OVERLAPS,'Price item','Charge','Discount', 'Comments (charge)','Group','Cost center name','Cost center code']

### work on a copy containing only bookings with overlaps
dfo = df[df[HAS_OVERLAPS] == True].copy()

# loop over bookings that have overlaps
for i in dfo.index:
    c = dfo.loc[i]
    #print(c[INTERVAL])
    # find the overlapping bookings: this would take a long time to run on the whole dataframe
    dfo[OVERLAPS] = dfo.apply(overlaps_another_booking, testrow=c, axis=1)
dfo[cols].to_excel(INVOICE_DIR / ("test_" + basename + "__overlapping_bookings.xlsx"), index=True)


### continue on the complete dataframe
# copy overlaps to complete dataframe (magically the indexing works)
df[OVERLAPS] = dfo[OVERLAPS]
df[cols].to_excel(DEBUG_DIR / ("tmp_" + basename + "__dfo2.xlsx"), index=True)


# test only with lauri
#df = df[df['User email'].str.contains('lauri')]

#for i in df.index:
#idx = df[df[HAS_OVERLAPS].index
#for i in idx:
#    c = df.loc[i]
#    print(c[INTERVAL])
#    df[OVERLAPS] = df.apply(overlaps_another_booking, testrow=c, axis=1)

#summary.append(OVERLAPS)
#df[df[OVERLAPS] != ''][summary]
#df[df[OVERLAPS] != ''][summary].to_excel(os.path.join(INVOICE_DIR,"test_" + basename + "__overlapping_bookings2.xlsx"), index=True)


## Save test files after fixes

In [ ]:
df['tmp_needs_split_3'] = df.apply(needs_split_3, axis=1)

summary.append('tmp_discount_factor')
run_tests("_after_fixes")

## Check total after fixes

In [ ]:
totals_wbs, df_wbs = check_totals(df,'after_fixes',INVOICE_DIR,basename)

## Save results

In [ ]:
# remove tmp_ columns
df = df[df.columns.drop(list(df.filter(regex='tmp_')))]

df = df.rename(columns={'Quantity':'Qty'})

# save discount as %, not a string
df['Discount'] = df['Discount'].str.replace('nan','0')
df['Discount'] = df['Discount'].str.replace(' %','')
df['Discount'] = df['Discount'].astype(float)
# save discount as fraction
df['Discount'] = df['Discount'] / 100

# sort by original ID
#df = df.sort_values(['ID','Booking start','Qty'], ascending=[True, True, False])

# sort by time (booking start)
df = df.sort_values(['Booking start',RESOURCE], ascending=[True, True])

# sort by time (creation date)
#df = df.sort_values(['Creation date',RESOURCE], ascending=[True, True])

# Append products
products = products.rename(columns={'Quantity':'Qty'})
df = pd.concat([df, products])
total_wbs, df_wbs = check_totals(df,'with_products',INVOICE_DIR,basename)

#
# Save summary with verifiers
# 

# extract verifiers, save excel for debugging
verifiers = df[['Cost center code',COL_REQUESTER_NAME]]\
    .groupby(['Cost center code'])[['Cost center code', COL_REQUESTER_NAME]]\
    .transform(lambda x: ', '.join(sorted(set(x))))\
    .reset_index()
verifiers = verifiers.drop(columns=['index'])
verifiers = verifiers.drop_duplicates(subset=['Cost center code'], keep='first')
verifiers.to_excel(DEBUG_DIR / ("test_" + basename + "__verifiers.xlsx"), index=False)

# create the summary
tmp = df.groupby(['Group','Remit code','Cost center code'])['Charge'].sum().reset_index()

# add verifiers
tmp = pd.merge(tmp, verifiers, on='Cost center code')

# save excel
tmp = tmp.rename({COL_REQUESTER_NAME:'Verifier name(s)'}, axis=1)
tmp.loc['Column_Total']= tmp.sum(numeric_only=True, axis=0)
tmp.to_excel(INVOICE_DIR / ("test_" + basename + "__totals_by_group_and_wbs_with_verifiers.xlsx"), index=False)
total_wbs = round(tmp.loc['Column_Total']['Charge'],2)
print("grouped by WBS: " + str(total_wbs))


# save full fixed version
df.to_excel(INVOICE_DIR / (basename + "_fixed" + ext), index=False)

# if there is no header, read it from disk
if header.empty:
    header = pd.read_excel(INVOICE_DIR / (basename + '__header.xlsx'))
    
# fix header total
header['Total'] = str(totals_wbs) + " EUR"

# save header
header.to_excel(INVOICE_DIR / (basename + "__header.xlsx"), index=False)

# save header and data as .csv
header.to_csv(DEBUG_DIR / (basename + "_fixed_header.csv"), index=False)
df.to_csv(DEBUG_DIR / (basename + "_fixed.csv"), mode='a', index=False)


In [ ]:
df.shape


In [ ]:
header


In [ ]:
test = df[df['Cost center code'].apply(lambda x: isinstance(x, int))]
save_test_result("test_" + basename + "__cost_center_code_int.xlsx", test, where=DEBUG_DIR)


In [ ]:
import openpyxl
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder

def as_text(value):
    if value is None:
        return ""
    return str(value)

def adjust_col_width(xlsx):
    wb = openpyxl.load_workbook(xlsx)
    worksheet = wb['Sheet1']
    
    for column_cells in worksheet.columns:
        length = max(len(as_text(cell.value)) for cell in column_cells)
        worksheet.column_dimensions[openpyxl.utils.get_column_letter(column_cells[0].column)].width = length
    
    for cell in worksheet['D']:
        cell.number_format = '####0.00€'
        cell.alignment = Alignment(horizontal='right')

    wb.save(filename=xlsx)
    
xlsx = INVOICE_DIR / ("test_" + basename + "__totals_by_group_and_wbs_with_products.xlsx")
print(xlsx)
adjust_col_width(xlsx)
xlsx = INVOICE_DIR / ("test_" + basename + "__totals_by_group_and_wbs_with_verifiers.xlsx")
print(xlsx)
adjust_col_width(xlsx)


In [ ]:
import re 

pohja = 'Sisäinen_lasku_monta_asiakasta_POHJA.xlsx'
ailille = INVOICE_DIR / pohja.replace('POHJA', basename)
print(ailille)

wb = openpyxl.load_workbook(pohja)
ws = wb['Sisäinen lasku, monta asiakasta']

ws['B10'] = 'H919'
ws['B12'] = 'Light Microscopy Unit'
ws['B14'] = 'Viikinkaari 5'
ws['B16'] = 'Mika Molin'

internal = "^H[0-9]{2,22}"
df_int = df[df['Remit code'].str.contains(internal, regex=True)]
GROUP = 'Group'
HEADS = 'Group head(s) text'
EMAIL = 'Email'
grp_and_heads = df_int.drop_duplicates(subset=[GROUP])[[GROUP, HEADS]]


# split group name to first and last name, replace ääkköset, construct email addresses
grp_and_heads[['Last', 'First']] = grp_and_heads[GROUP].str.rsplit(' ', 1, expand=True)
tdict = {'ä':'a', 'ö':'o', 'å':'a'}
ttable = "äöå".maketrans(tdict)
grp_and_heads['First'] = grp_and_heads['First'].str.translate(ttable)
grp_and_heads['Last'] = grp_and_heads['Last'].str.translate(ttable)
grp_and_heads['first'] = grp_and_heads['First'].str.lower()
grp_and_heads['fast'] = grp_and_heads['Last'].str.lower()
grp_and_heads['First.Last'] = grp_and_heads['First'] + '.' + grp_and_heads['Last'] + '@helsinki.fi'
grp_and_heads['first.last'] = grp_and_heads['first'] + '.' + grp_and_heads['fast'] + '@helsinki.fi'

# initialize Contact as all Group heads
grp_and_heads['Contact'] = grp_and_heads[HEADS]

# function to remove constructed email addresses from group heads
def subtract_email(row):
    heads = str(row[HEADS])
    email = str(row['first.last'])
    Email = str(row['First.Last'])
    #print(heads)
    #print(email)
    heads = heads.replace(email,'')
    heads = heads.replace(Email,'')
    # this will mess up cases with two added heads (e.g. Alitalo), majority looks ok
    heads = heads.replace(' ','')
    heads = heads.replace(';','')
    #heads = heads.replace('; ','')
    heads = re.sub('helsinki.fi?', 'helsinki.fi; ', heads)
    
    return heads

grp_and_heads['AddedContact'] = grp_and_heads.apply(subtract_email, axis=1)
added_contacts = grp_and_heads[(grp_and_heads['AddedContact'].str.len()>0)]
for i in added_contacts.index:
    grp_and_heads.at[i,'Contact'] = added_contacts.at[i,'AddedContact']

def get_contact(table, group):
    contact = table[table['Group']==group]['Contact'].values[0]
    return str(contact)

#print(get_contact(grp_and_heads, 'Ojala Päivi'))


internal = re.compile("H[0-9]{2,22}")

# start filling in invoice lines from this row
i = 9
for index, row in df_wbs.iterrows():
    if not internal.match(str(row['Remit code'])):
        continue
        
    ws['J' + str(i)] = row['Remit code']
    ws['K' + str(i)] = row['Group']
    ws['L' + str(i)] = get_contact(grp_and_heads, row['Group'])
    ws['M' + str(i)] = row['Cost center code']
    ws['Q' + str(i)] = "%s %s.xlsx" % (row['Group'], row['Cost center code'])
    ws['R' + str(i)] = row['Charge']
    
    i = i+1

wb.save(filename=ailille)


In [ ]:
grp_and_heads